---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, Imputer
#from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, auc
import datetime
import re

In [101]:
def blight_model():
    
    def read_data():
        X = pd.read_csv('train.csv', encoding =  'ISO-8859-1', index_col = 'ticket_id')
        X = X.dropna(subset=['compliance'])
        X_test = pd.read_csv('test.csv', encoding =  'ISO-8859-1', index_col = 'ticket_id')

        y_col = ['payment_amount', 'balance_due', 'payment_date', 'payment_status', 
            'collection_status', 'compliance_detail', 'compliance']
        y = X['compliance'].copy()
        X = X.drop(y_col, axis =1)
        return X, y, X_test
    
    def data_cat(df):
        data_type = [(col, df[col].dtype) for col in df.columns]
        geo_cols =  ['violation_street_number','violation_street_name','violation_zip_code',
                 'mailing_address_str_number', 'mailing_address_str_name', 'city','state','zip_code', 
                 'non_us_str_code', 'country']
        date_cols = ['ticket_issued_date','hearing_date']
        data_cols = [col for col in df.columns if not col in (geo_cols + date_cols)]
        return geo_cols, date_cols, data_cols


    def date_parse(df, date_cols):
        time_X = df[date_cols].copy()
        time_X['ticket_issued_date_parsed'] = pd.to_datetime(time_X['ticket_issued_date'], infer_datetime_format=True)
        time_X['hearing_date_parsed'] = pd.to_datetime(time_X['hearing_date'], infer_datetime_format=True)
        time_X['ticket_issued_year'] = time_X['ticket_issued_date_parsed'].dt.year
        time_X['ticket_issued_month'] = time_X['ticket_issued_date_parsed'].dt.month
        time_X['ticket_issued_day'] = time_X['ticket_issued_date_parsed'].dt.day
        time_X['ticket_issued_hour'] = time_X['ticket_issued_date_parsed'].dt.hour

        time_X['hearing_date_year'] = time_X['hearing_date_parsed'].dt.year
        time_X['hearing_date_month'] = time_X['hearing_date_parsed'].dt.month
        time_X['hearing_date_day'] = time_X['hearing_date_parsed'].dt.day
        time_X['hearing_date_hour'] = time_X['hearing_date_parsed'].dt.hour

        time_X = time_X.drop(['ticket_issued_date_parsed','hearing_date_parsed'], axis=1)
        time_X = time_X.drop(date_cols, axis=1)
        return time_X

    def geo_parse(df, geo_cols):
        address = pd.read_csv('addresses.csv', index_col='ticket_id')
        latlong = pd.read_csv('latlons.csv')

        # merge
        geo_X = df[geo_cols].copy()
        geo_X_2 = pd.merge(geo_X,address, how='left', left_index=True, right_index=True)
        geo_X_3 = pd.merge(geo_X_2, latlong, how='left', left_on='address', right_on='address')
        geo_X_3.index = df.index
    
        geo_str_list = ['violation_street_name','mailing_address_str_name','city', 
                        'state', 'country']
        for i in geo_str_list:
            geo_X_3[i] = geo_X_3[i].str.lower()
            geo_X_3[i] = geo_X_3[i].str.strip()
        return geo_X_3

    def other_data_parse(df, data_cols):
        other_X = df[data_cols].copy()
        for i in [col for col in other_X.columns if other_X[col].dtype =='object']:
            other_X[i] = other_X[i].str.lower()
            other_X[i] = other_X[i].str.strip()
        return other_X

    def df_merge(df, geo_cols, date_cols, data_cols):
        time_X = date_parse(df, date_cols)
        geo_X_3 = geo_parse(df, geo_cols)
        other_X = other_data_parse(df, data_cols)
        other_geo_X = pd.merge(other_X, geo_X_3, how='inner', left_index=True, right_index=True)
        final_X = pd.merge(other_geo_X, time_X, how='inner', left_index=True, right_index=True)
        return final_X

    def col_clean(df):
        df = df.drop(['grafitti_status','non_us_str_code','violation_zip_code', 'zip_code'], axis=1)
        return df

    def geo_feature(df):
        X_train = df.copy()
        X_train.loc[:,'mail_loc'] = 0
        X_train.loc[(X_train['country']=='usa')&(X_train['state']=='mi')&(X_train['city']=='detroit'),['mail_loc']] = 3
        X_train.loc[(X_train['country']=='usa')&(X_train['state']=='mi')&(X_train['city']!='detroit'),['mail_loc']] = 2
        X_train.loc[(X_train['country']=='usa')&(X_train['state']!='mi'),['mail_loc']] = 1
        X_train.loc[(X_train['country']!='usa'),['mail_loc']]=0

    # drop useless column
        X_train = X_train.drop(['country', 'city','state', 'violation_street_name', 'mailing_address_str_number', 
                                'violator_name', 'mailing_address_str_name', 'address'], axis=1)
        return X_train

    def clean_feature(df):
        df = df.drop(['clean_up_cost'],axis=1) #'admin_fee','state_fee',
        return df

    def freq_coder(df, freq_col, fit):
        for i in freq_col:
            feq_map = fit.groupby(i).size()/len(fit)
            df.loc[:,i] = df[i].map(feq_map)
            df[i] = df[i].fillna(0)
        return df

    def oh_coder(df,onehot_col):
        oh_df = df[onehot_col].copy()
        oh_df_code=pd.get_dummies(oh_df)
        oh_coded_df = pd.concat([df, oh_df_code], axis=1)
        oh_coded_df = oh_coded_df.drop([col for col in onehot_col],axis=1)

        return oh_coded_df

    X, y, X_test = read_data()
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)
    geo_cols, date_cols, data_cols = data_cat(X_train)
    
    X_train_1 = df_merge(X_train, geo_cols, date_cols, data_cols)
    X_valid_1 = df_merge(X_valid, geo_cols, date_cols, data_cols)
    X_test_1 = df_merge(X_test, geo_cols, date_cols, data_cols)    

    X_train_3 = X_train_1.pipe(col_clean).pipe(geo_feature).pipe(clean_feature)
    X_valid_3 = X_valid_1.pipe(col_clean).pipe(geo_feature).pipe(clean_feature)
    X_test_3 = X_test_1.pipe(col_clean).pipe(geo_feature).pipe(clean_feature)

    # encode
    freq_col = ['inspector_name', 'violation_code', 'violation_description'] ##'agency_name', 'disposition',
    label_col = ['agency_name', 'disposition','mail_loc']
    onehot_col = ['agency_name', 'disposition']
    
    # freq encode
    X_train_3 = freq_coder(X_train_3, freq_col=freq_col, fit=X_train_3)
    X_valid_3 = freq_coder(X_valid_3, freq_col=freq_col, fit=X_train_3)
    X_test_3 = freq_coder(X_test_3, freq_col=freq_col, fit=X_train_3)

    # label encode
    labelencoder = LabelEncoder()
    for i in label_col:
        labelencoder.fit(X_train_3[i])
        X_train_3[i] = X_train_3[i].map(lambda s: 0 if s not in labelencoder.classes_ else s)
        X_valid_3[i] = X_valid_3[i].map(lambda s: 0 if s not in labelencoder.classes_ else s)
        X_test_3[i] = X_test_3[i].map(lambda s: 0 if s not in labelencoder.classes_ else s)
    
    # onehot encode
    X_train_3 = oh_coder(X_train_3,onehot_col=onehot_col)
    X_valid_3 = oh_coder(X_valid_3,onehot_col=onehot_col)
    X_test_3 = oh_coder(X_test_3,onehot_col=onehot_col)

    # impute
    imputer = Imputer(missing_values=np.nan, strategy='most_frequent')
    missing_col = [col for col in X_train_3.columns if X_train_3[col].isnull().sum()>0]
    for col in missing_col:
        X_train_3[col] = imputer.fit_transform(X_train_3[col].values.reshape(-1,1))
        X_valid_3[col] = imputer.transform(X_valid_3[col].values.reshape(-1,1))
        X_test_3[col] = imputer.transform(X_test_3[col].values.reshape(-1,1))
    
    # model building
    fea = ['inspector_name','late_fee','discount_amount','hearing_date_month',
        'lat','lon','disposition_responsible (fine waived) by deter','disposition_responsible by admission',
           'disposition_responsible by default']

    # fea = ['inspector_name','violation_code','violation_description','disposition',
    #        'fine_amount','admin_fee','state_fee','late_fee','discount_amount','judgment_amount', 'violation_street_number',
    #        'lat','lon','ticket_issued_year','ticket_issued_month','ticket_issued_day','ticket_issued_hour',
    #        'hearing_date_year','hearing_date_month','hearing_date_day','hearing_date_hour','mail_loc'
    #        'agency_name_buildings, safety engineering & env department','agency_name_department of public works',
    #       'agency_name_detroit police department','agency_name_health department', 'agency_name_neighborhood city halls',
    #       'disposition_responsible (fine waived) by deter','disposition_responsible by admission','disposition_responsible by default',
    #       'disposition_responsible by determination']

    X_train_feature = X_train_3[fea].copy()
    X_valid_feature = X_valid_3[fea].copy()

    gbm4 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, max_depth=15, min_samples_leaf=30, min_samples_split=400,
                                      random_state=10,max_features='sqrt',subsample=0.8)

    # parameters are optimized by GridSearch, report enclosed below, the GridSearch is commented due to kernel workload

    gbm4.fit(X_train_feature,y_train)
    # grid_gbm4.best_score_ = 0.838277577044181

    prob = gbm4.predict_proba(X_test_3[fea].copy())
    compliant_prob = [p[1] for p in prob]
    
    return pd.Series(compliant_prob, index=X_test_3.index)

In [85]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


ticket_id
284932    0.067803
285362    0.029673
285361    0.053015
285338    0.176465
285346    0.064292
285345    0.096571
285347    0.125054
285342    0.946543
285530    0.052755
284989    0.092368
285344    0.086665
285343    0.036699
285340    0.046615
285341    0.156779
285349    0.069640
285348    0.104305
284991    0.092368
285532    0.124565
285406    0.077466
285001    0.094163
285006    0.028364
285405    0.029729
285337    0.092347
285496    0.052026
285497    0.073191
285378    0.036934
285589    0.058355
285585    0.076062
285501    0.062260
285581    0.040236
            ...   
376367    0.101854
376366    0.132765
376362    0.183153
376363    0.135898
376365    0.101854
376364    0.132765
376228    0.168309
376265    0.128703
376286    0.136996
376320    0.170859
376314    0.128664
376327    0.358416
376385    0.211362
376435    0.961743
376370    0.969914
376434    0.060242
376459    0.104727
376478    0.027142
376473    0.154204
376484    0.120231
376482    0.138969
37

In [76]:
# baseline model evaluation

#gbm0 = GradientBoostingClassifier(random_state=10)
#gbm0.fit(X_train_feature, y_train)

# y_valid_pred = gbm0.predict(X_valid_feature)
#y_valid_score = gbm0.decision_function(X_valid_feature)
#cv_score = cross_val_score(gbm0, X_train_feature, y_train, cv=5, scoring='roc_auc')
#fp,tp,_ = roc_curve(y_valid, y_valid_score)
#rocauc = auc(fp, tp)
#print("Accuracy score (training): {0:.3f}".format(gbm0.score(X_train_feature, y_train)))
#print("Accuracy score (validation): {0:.3f}".format(gbm0.score(X_valid_feature, y_valid)))
#print("CV Score : Mean - {} | Std - {} | Min - {} | Max - {}".format(np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
#print("Confusion Matrix:")
#print(confusion_matrix(y_valid, y_valid_pred))
#print("Classification Report")
#print(classification_report(y_valid, y_valid_pred))
#print("Area under ROC curve = {:0.2f}".format(rocauc))

#Accuracy score (training): 0.944
#Accuracy score (validation): 0.900
#CV Score : Mean - 0.8231856519494904 | Std - 0.005674471690844688 | Min - 0.8120439348796218 | Max - 0.8273769606669727
#Confusion Matrix:
#[[34580  2498]
# [ 1499  1393]]
#Classification Report
#              precision    recall  f1-score   support

#         0.0       0.96      0.93      0.95     37078
#         1.0       0.36      0.48      0.41      2892

#    accuracy                           0.90     39970
#   macro avg       0.66      0.71      0.68     39970
#weighted avg       0.92      0.90      0.91     39970

#Area under ROC curve = 0.78

In [77]:
#gbm1 = GradientBoostingClassifier(learning_rate=0.1, min_samples_split = 500, min_samples_leaf = 50, random_state=10, max_depth=8, max_features='sqrt',subsample=0.8)
#param_grid = {'n_estimators':range(20,81,10)}
#grid_gbm1 = GridSearchCV(gbm1, param_grid=param_grid, scoring='roc_auc', n_jobs=4, cv=5)
#grid_gbm1.fit(X_train_feature, y_train)
####grid_gbm1.best_params_, grid_gbm1.best_score_
####({'n_estimators': 80}, 0.8328128440829934)

In [78]:
#gbm2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, random_state=10, max_features='sqrt',subsample=0.8)
#param_grid = {'max_depth':range(5,16,2), 'min_samples_split':range(400,1201,200)}
#grid_gbm2 = GridSearchCV(gbm2, param_grid=param_grid, scoring='roc_auc', n_jobs=4, cv=5)
#grid_gbm2.fit(X_train_feature, y_train)
####grid_gbm2.best_params_, grid_gbm2.best_score_
####({'max_depth': 15, 'min_samples_split': 400}, 0.8383864052451784)

In [79]:
#gbm3 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, max_depth=15, random_state=10, max_features='sqrt',subsample=0.8)
#param_grid = {'min_samples_split':range(400,1100,200), 'min_samples_leaf':range(30,71,10)}
#grid_gbm3 = GridSearchCV(gbm3, param_grid=param_grid, scoring='roc_auc', n_jobs=4, cv=5)
#grid_gbm3.fit(X_train_feature, y_train)
#### grid_gbm3.best_params_, grid_gbm3.best_sc
#### ({'min_samples_leaf': 30, 'min_samples_split': 400}, 0.8379585792222688)

In [90]:
#gbm4 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, max_depth=15, min_samples_leaf=30, min_samples_split=400, random_state=10,subsample=0.8)
#param_grid = {'max_features':range(7,20,2)}
#grid_gbm4 = GridSearchCV(gbm4, param_grid=param_grid, scoring='roc_auc', n_jobs=4, cv=5)
#grid_gbm4.fit(X_train_feature, y_train)
#### grid_gbm4.best_params_, grid_gbm4.best_score_
#### ({'max_features': 19}, 0.838277577044181)